In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

In [7]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")
train

,RowId,IntersectionId,Latitude,Longitude,EntryStreetName,ExitStreetName,EntryHeading,ExitHeading,Hour,Weekend,...,TimeFromFirstStop_p40,TimeFromFirstStop_p50,TimeFromFirstStop_p60,TimeFromFirstStop_p80,DistanceToFirstStop_p20,DistanceToFirstStop_p40,DistanceToFirstStop_p50,DistanceToFirstStop_p60,DistanceToFirstStop_p80,City
0,1921357,0,33.791659,-84.430032,Marietta Boulevard Northwest,Marietta Boulevard Northwest,NW,NW,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Atlanta
1,1921358,0,33.791659,-84.430032,Marietta Boulevard Northwest,Marietta Boulevard Northwest,SE,SE,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Atlanta
2,1921359,0,33.791659,-84.430032,Marietta Boulevard Northwest,Marietta Boulevard Northwest,NW,NW,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Atlanta
3,1921360,0,33.791659,-84.430032,Marietta Boulevard Northwest,Marietta Boulevard Northwest,SE,SE,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Atlanta
4,1921361,0,33.791659,-84.430032,Marietta Boulevard Northwest,Marietta Boulevard Northwest,NW,NW,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Atlanta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
856382,2777739,1990,39.938232,-75.149930,South 4th Street,Catharine Street,S,W,17,1,...,0.0,0.0,0.0,55.0,0.0,0.0,0.0,0.0,75.5,Philadelphia
856383,2777740,1990,39.938232,-75.149930,South 4th Street,South 4th Street,S,S,18,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Philadelphia
856384,2777741,1990,39.938232,-75.149930,South 4th Street,South 4th Street,S,S,19,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Philadelphia
856385,2777742,1990,39.938232,-75.149930,Catharine Street,Catharine Street,W,W,20,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Philadelphia


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 856387 entries, 0 to 856386
Data columns (total 28 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   RowId                    856387 non-null  int64  
 1   IntersectionId           856387 non-null  int64  
 2   Latitude                 856387 non-null  float64
 3   Longitude                856387 non-null  float64
 4   EntryStreetName          848239 non-null  object 
 5   ExitStreetName           850100 non-null  object 
 6   EntryHeading             856387 non-null  object 
 7   ExitHeading              856387 non-null  object 
 8   Hour                     856387 non-null  int64  
 9   Weekend                  856387 non-null  int64  
 10  Month                    856387 non-null  int64  
 11  Path                     856387 non-null  object 
 12  TotalTimeStopped_p20     856387 non-null  float64
 13  TotalTimeStopped_p40     856387 non-null  float64
 14  Tota

## One-hot encoding

In [11]:
str_col = ['EntryStreetName', 'ExitStreetName', 'EntryHeading', 'ExitHeading', 'Path', 'City']
train_onehot = train.copy()
test_onehot = test.copy()
for col in str_col:
    head = train[col].value_counts().head(80).index.tolist()
    train_onehot[col] = train_onehot[col].apply(lambda x: 'pleaseRemove' if head.count(x) == 0 else x)
    test_onehot[col] = test_onehot[col].apply(lambda x: 'pleaseRemove' if head.count(x) == 0 else x)
    print(col)

# print(train['ExitStreetName'].value_counts())
# print(train['EntryHeading'].value_counts())
# print(train['ExitHeading'].value_counts())
# print(train['Path'].value_counts())
# print(train['City'].value_counts())

EntryStreetName
ExitStreetName
EntryHeading
ExitHeading
Path
City


In [13]:
X = pd.get_dummies(train_onehot, columns=['EntryStreetName', 'ExitStreetName', 'EntryHeading', 'ExitHeading', 'Path', 'City'])
Y = train[['TotalTimeStopped_p20', 'TotalTimeStopped_p50', 'TotalTimeStopped_p80', 'DistanceToFirstStop_p20', 'DistanceToFirstStop_p50', 'DistanceToFirstStop_p80']]

for col in X.columns.tolist():
    if col.find('pleaseRemove') != -1:
        X.drop(col, axis = 1, inplace = True)
    if col.find('Stop') != -1:
        X.drop(col, axis = 1, inplace = True)

In [15]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 856387 entries, 0 to 856386
Columns: 267 entries, RowId to City_Philadelphia
dtypes: float64(2), int64(5), uint8(260)
memory usage: 258.1 MB


In [17]:
X.columns

Index(['RowId', 'IntersectionId', 'Latitude', 'Longitude', 'Hour', 'Weekend',
       'Month', 'EntryStreetName_10th Street Northwest',
       'EntryStreetName_17th Street Northwest',
       'EntryStreetName_Aramingo Avenue',
       ...
       'Path_West Lehigh Avenue_E_West Lehigh Avenue_E',
       'Path_West Lehigh Avenue_W_West Lehigh Avenue_W',
       'Path_West North Avenue_E_West North Avenue_E',
       'Path_West North Avenue_W_West North Avenue_W',
       'Path_West Roosevelt Road_E_West Roosevelt Road_E',
       'Path_West Roosevelt Road_W_West Roosevelt Road_W', 'City_Atlanta',
       'City_Boston', 'City_Chicago', 'City_Philadelphia'],
      dtype='object', length=267)

In [ ]:
predict_X = pd.get_dummies(test_onehot, columns=['EntryStreetName', 'ExitStreetName', 'EntryHeading', 'ExitHeading', 'Path', 'City'])

In [ ]:
for col in predict_X.columns.tolist():
    if col.find('pleaseRemove') != -1:
        predict_X.drop(col, axis = 1, inplace = True)

In [ ]:
predict_X.info()

In [29]:
ml = RandomForestRegressor()
ml.fit(X, Y)

KeyboardInterrupt: 

In [ ]:
predict_Y = ml.predict(predict_X)
predict_Y

In [ ]:
predict_Y.size

In [ ]:
submission = pd.DataFrame(data = {'TargetId': [], 'Target': []})
for target_ID, v in enumerate(predict_Y):
    for index, j in enumerate(v):
        target_id = str(target_ID) + "_" + str(index)
        submission.append({'TargetId': t
        arget_id, 'Target': j}, ignore_index=True)

submission.to_csv("submission.csv", index=False)

In [ ]:
train.columns